This notebook demonstrates the procedure presented in

P. Bestagini, F. Lombardi, M. Lualdi, F. Picetti, S. Tubaro, <i>Landmine Detection Using Autoencoders on Multi-polarization GPR Volumetric Data</i>, Oct. 2018

In [1]:
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import PreProcessing
import net
from PatchExtractor import PatchExtractor

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau



GPU selected: 0


c:\Users\mihau\anaconda3\envs\tf_36\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\mihau\anaconda3\envs\tf_36\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\mihau\anaconda3\envs\tf_36\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\mihau\anaconda3\envs\tf_36\lib\site-packages\tensorflow\python\f

# Variables

In [2]:
in_path = 'giuriati_2/20170621_deg0_HHVV.npy'
out_path = 'cnn_article'
architecture = 'Auto3D2'
ny = 3 # number of adjacent B-scans to be considered
data_augmentation = True
preprocessing = 'normalize'
patch_size = 64
patch_stride = 4
n_bsc = 5 # number of B_scans for training

In [3]:
def parse_pp(string):
    return getattr(PreProcessing, string)

def parse_net(string):
    return getattr(net, string)

In [4]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

field, campaign = in_path.split('/')
campaign, extension = campaign.split('.')

# Load Datasets

In [5]:

dataset = np.load('./datasets/'+str(in_path),allow_pickle=True).item()

train_bsc_idx = np.where(np.asarray(dataset['ground_truth']) == 0)[0][:n_bsc]
trainset = dataset['data'][train_bsc_idx]
trainset = np.moveaxis(trainset, np.argmin(trainset.shape), -1)
del dataset

### block extraction

In [6]:
if patch_size is not None:
    patch_size = (patch_size, patch_size)
else:
    patch_size = trainset.shape[1:]

patch_size = patch_size + (ny,)
patch_stride = (patch_stride, patch_stride, 1)

pe = PatchExtractor(patch_size, stride=patch_stride)

train_patches = pe.extract(trainset)
# reshaping
train_patches = train_patches.reshape((-1,) + patch_size)

# preprocessing each patch
train_patches, min_tr, max_tr = PreProcessing.apply_transform(train_patches, transform=parse_pp(preprocessing))

# Data augmentation (default=True)
if data_augmentation:
    train_patches = np.concatenate([train_patches, np.flip(train_patches, axis=2).copy()], axis=0)

train_patches = shuffle(train_patches)

# create training and validation sets
train_patches, val_patches, train_index, val_index = train_test_split(train_patches,
                                                                      np.arange(train_patches.shape[0]),
                                                                      test_size=0.5,
                                                                      random_state=118
                                                                      )

# CNN Architecture

In [7]:
sets = net.Settings()
patience = sets.patience
lr_factor = sets.lr_factor
batch_size = sets.batch_size
epochs = sets.epochs

autoencoder, encoder = parse_net(architecture)(patch_size)

out_name = field+'_'+campaign+'_'+architecture+'_patch'+str(patch_size[0])+'_stride'+str(patch_stride[0])+'_bsc'+str(n_bsc)+'_ny'+str(ny)

lr_chkpt = ReduceLROnPlateau(monitor='val_loss',
                             factor=lr_factor,
                             patience=patience//2,
                             verbose=0,
                             mode='auto',
                             epsilon=0.0001,
                             cooldown=0,
                             min_lr=0)
save_chkpt = ModelCheckpoint(os.path.join(out_path, out_name+'.h5'),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')
stop_chkpt = EarlyStopping(monitor='val_loss',
                           patience=patience)

In [8]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_in_0 (InputLayer)          (None, 64, 64, 3)         0         
_________________________________________________________________
g_conv_0 (Conv2D)            (None, 64, 64, 16)        1744      
_________________________________________________________________
g_conv_1 (Conv2D)            (None, 32, 32, 16)        6416      
_________________________________________________________________
g_conv_2 (Conv2D)            (None, 16, 16, 16)        4112      
_________________________________________________________________
g_conv_3 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 16)          1040      
_________________________________________________________________
encoder (Conv2D)             (None, 2, 2, 16)          272       
__________

### Training

In [9]:
train = autoencoder.fit(train_patches, train_patches,
                        validation_data=(val_patches, val_patches),
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        callbacks=[save_chkpt, stop_chkpt, lr_chkpt])
print('Training done!')

Epoch 00000: val_loss improved from inf to 0.01410, saving model to cnn_article\giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3.h5
Training done!


# Deployment (test)

In [10]:
from sklearn.metrics import roc_curve, roc_auc_score
from tqdm import tqdm

In [11]:
training = './cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3'
net_weights = training + '.h5'

### Load dataset

In [12]:
# in this case, the dataset for training in the same for testing
train_path = in_path
dataset = np.load('./datasets/' + str(in_path),allow_pickle=True).item()

# preprocessing
data = dataset['data']

# patch extractor
pe = PatchExtractor(patch_size, stride=patch_stride)

# background bscans for training
gt = np.asarray(dataset['ground_truth'])
del dataset

test_idx = np.arange(data.shape[0])
# check whether the test dataset is the same of the training
if in_path == train_path:
    train_idx = np.where(gt == 0)[0][:n_bsc]
    test_idx = np.delete(test_idx, train_idx)
testset = data[test_idx]
gt = gt[test_idx]
del data
testset = np.moveaxis(testset, np.argmin(testset.shape), -1)

### Load architecture

In [13]:
autoencoder, encoder = parse_net(architecture)(patch_size)
autoencoder.load_weights(os.path.join(net_weights))

out_name = field+'_'+campaign+'_'+architecture+'_patch'+str(patch_size[0])+'_stride'+str(patch_stride[0])+'_bsc'+str(n_bsc)+'_ny'+str(ny)

### Test loop

In [14]:
patches = pe.extract(testset)
del testset
patchesIdx = patches.shape
patches_hat = autoencoder.predict(patches.reshape((-1,) + patch_size))
mseFeat = (encoder.predict(patches.reshape((-1,) + patch_size)) - encoder.predict(patches_hat))**2
mseFeat_patches = np.zeros(patches_hat.shape) + np.mean(mseFeat, axis=(1,2,3)).reshape((-1,1,1,1))
del patches
del patches_hat
del mseFeat
mseFeat_vol = pe.reconstruct(mseFeat_patches.reshape(patchesIdx))
del mseFeat_patches

MemoryError: Unable to allocate 13.9 GiB for an array with shape (151335, 64, 64, 3) and data type float64

### Evaluation

In [15]:
mse_mask_max = np.max(mseFeat_vol, axis=(0, 1))
fpr_max, tpr_max, thresholds_max = roc_curve(gt, mse_mask_max)
roc_auc_max = roc_auc_score(gt, mse_mask_max)
print('best AUC = %0.2f' % roc_auc_max)

NameError: name 'mseFeat_vol' is not defined